### Make a surface and domain file given a lat  lon 

In [1]:
from scipy.stats import qmc
import numpy as np
import xarray as xr
import csv
import pandas as pd
import os
import netCDF4 as nc4
import sys
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                     
import shutil                                                                                                                                                       
import tempfile 
import random


### Domain file

In [2]:
lat = 9.154300000000
lon = -79.846100000000

if lon < 0 : 
    lon = 360 + lon
    print(lon)

280.1539


In [3]:
domain_full = '/global/cfs/cdirs/e3sm/inputdata/share/domains/domain.lnd.fv1.9x2.5_gx1v6.090206.nc'
domain_full = xr.open_dataset(domain_full,  decode_times=False)

In [5]:
lon_mids = domain_full.xc[0,0:144]
lat_mids = domain_full.yc[0:96,0]

abslat = np.abs(lat_mids - lat)
abslon = np.abs(lon_mids - lon)
c = np.maximum(abslon, abslat)


([xloc],[yloc]) = np.where(c == np.min(c))

lat_new = lat_mids[yloc]
print(lat_new.values)
lon_new = lon_mids[xloc]
print(lon_new.values)

domain = domain_full.sel(ni=[xloc],nj=[yloc])

8.5263157894736
279.99999999999994


#### Set frac to 1

In [7]:
domain['frac'].values = [[1.0]]

[[1.]]


#### Save domain file

In [9]:
domain.to_netcdf('/global/cfs/cdirs/m2420/fates-tutorial-2024/fates-tutorial/met_data/bci/domain_bci_fates_tutorial.nc')

### Surface data file

In [10]:
surfdata_full = '/global/cfs/cdirs/e3sm/inputdata/lnd/clm2/surfdata_map/surfdata_1.9x2.5_simyr2000_c180306.nc'
surfdata_full = xr.open_dataset(surfdata_full,  decode_times=False)

<bound method ImplementsDatasetReduce._reduce_method.<locals>.wrapped_func of <xarray.Dataset>
Dimensions:             (lsmlat: 96, lsmlon: 144, nlevsoi: 10, natpft: 17, time: 12, lsmpft: 17, numurbl: 3, numrad: 2, nlevurb: 5)
Coordinates:
  * natpft              (natpft) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16
  * time                (time) int32 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: lsmlat, lsmlon, nlevsoi, lsmpft, numurbl, numrad, nlevurb
Data variables: (12/77)
    mxsoil_color        int32 ...
    mxsoil_order        int32 ...
    SOIL_COLOR          (lsmlat, lsmlon) int32 ...
    SOIL_ORDER          (lsmlat, lsmlon) int32 ...
    PCT_SAND            (nlevsoi, lsmlat, lsmlon) float64 ...
    PCT_CLAY            (nlevsoi, lsmlat, lsmlon) float64 ...
    ...                  ...
    PCT_URBAN           (numurbl, lsmlat, lsmlon) float64 ...
    URBAN_REGION_ID     (lsmlat, lsmlon) int32 ...
    APATITE_P           (lsmlat, lsmlon) float64 ...
    LABILE_P 

##### Subset by lat and lon

In [11]:
surfdata = surfdata_full.sel(lsmlon=[xloc],lsmlat=[yloc])

#### Set percent vegetation to 100%

In [14]:
new_urban = [[[0]],[[0]],[[0]]]
new_urban_array = xr.DataArray(new_urban, dims=['numurbl', 'lsmlat', 'lsmlon'],
                               coords = {'numurbl':surfdata['numurbl'],'lsmlat':surfdata['lsmlat'],'lsmlon':surfdata['lsmlon']})
surfdata['PCT_URBAN'] = new_urban_array

surfdata['PCT_LAKE'].values = [[0.0]]
surfdata['PCT_NATVEG'].values = [[100.0]]

#### Save surface file

In [16]:
surfdata.to_netcdf('/global/cfs/cdirs/m2420/fates-tutorial-2024/fates-tutorial/met_data/bci/surfdata_bci_fates_tutorial.nc')